In [ ]:
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
from PIL import Image
import os
from tqdm import tqdm, tqdm_notebook
import csv
import numpy as np
from differential_color_functions import rgb2lab_diff, ciede2000_diff
from perc_cw import PerC_CW
from perc_al import PerC_AL
import differential_color_functions
# get the list of images along with the specified target labels
def load_ground_truth(csv_filename):
    image_id_list = []
    label_tar_list = []

    with open(csv_filename) as csvfile:
        reader = csv.DictReader(csvfile, delimiter=',')
        for row in reader:
            image_id_list.append( row['ImageId'] )
            label_tar_list.append( int(row['TargetClass'])-1 )

    return image_id_list,label_tar_list

# simple Module to normalize an image
class Normalize(nn.Module):
    def __init__(self, mean, std):
        super(Normalize, self).__init__()
        self.mean = torch.Tensor(mean)
        self.std = torch.Tensor(std)
    def forward(self, x):
        return (x - self.mean.type_as(x)[None,:,None,None]) / self.std.type_as(x)[None,:,None,None]

# values are standard normalization for ImageNet images, from https://github.com/pytorch/examples/blob/master/imagenet/main.py
norm = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
trn = transforms.Compose([
     transforms.ToTensor(),])

# fix the random seed of pytorch and make cudnn deterministic for reproducing the same results
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True

In [ ]:
#load image list
image_id_list,label_tar_list=load_ground_truth('images.csv')
# specify the device    
device  = torch.device("cuda:0")
# load the pre-trained model
model = models.inception_v3(pretrained=True,transform_input=False).eval()
for param in model.parameters():
    param.requires_grad=False
model.to(device)

In [ ]:
## implement PerC_C&W and PerC_AL


#set the mode (untargeted or targeted)
untargeted=1
batch_size=20
num_batches = np.int(np.ceil(len(image_id_list)/batch_size))
color_differences=[]
l2_norm=[]
l_inf_norm=[]
output_path='images_adv_PerC_AL'
if os.path.exists (output_path)==False:
    os.mkdir(output_path)
for k in tqdm_notebook(range(0,num_batches)):
    batch_size_cur=min(batch_size,len(image_id_list)-k*batch_size)
    #load a batch of input images with the size of batch_size*channel*height*width
    X_ori = torch.zeros(batch_size_cur,3,299,299).to(device)    
    for i in range(batch_size_cur):  
        X_ori[i]=trn(Image.open(os.path.join('images',image_id_list[k*batch_size+i])+'.png'))  
    X_ori_LAB=rgb2lab_diff(X_ori,device)
    
    if untargeted:
        labels=torch.argmax(model((X_ori-0.5)/0.5),dim=1)
    
    else:
        labels=torch.tensor(label_tar_list[k*batch_size:k*batch_size+batch_size_cur]).to(device)

#     approach = PerC_CW(device=device,search_steps=9,max_iterations=1000,learning_rate=0.01,initial_const=10)
    approach = PerC_AL(device=device,max_iterations=1000,alpha_l_init=1,alpha_c_init=0.5,confidence=40)

    X_adv = approach.adversary(model, X_ori, labels=labels, targeted=False)

    color_distance_map=ciede2000_diff(X_ori_LAB,rgb2lab_diff(X_adv,device),device)   
    #claculate perceptual color_differences, L2 norm and L_inf norm
    color_differences.append(torch.norm(color_distance_map.view(batch_size_cur,-1),dim=1).cpu().numpy())
    l2_norm.append(torch.norm((X_adv-X_ori).view(batch_size_cur,-1),dim=1).cpu().numpy())
    l_inf_norm.append(torch.norm((X_adv-X_ori).view(batch_size_cur,-1),dim=1, p=float('inf')).cpu().numpy())
    
    #save the modified images
    for j in range(batch_size_cur):
        x_np=transforms.ToPILImage()(X_adv[j].detach().cpu())
        x_np.save(os.path.join(output_path,image_id_list[k*batch_size+j])+'.png') 

In [ ]:
## Evaluation on adversarial strength (success rate)

batch_size=20
num_batches = np.int(np.ceil(len(image_id_list)/batch_size))
cnt=0.
for k in tqdm_notebook(range(0,num_batches)):
    batch_size_cur=min(batch_size,1000-k*batch_size)
    X_ori = torch.zeros(batch_size_cur,3,299,299).to(device) 
    X_adv=torch.zeros_like(X_ori)
    for i in range(batch_size_cur): 
        X_ori[i]=trn(Image.open('./images/'+image_id_list[k*batch_size+i]+'.png'))
        X_adv[i]=trn(Image.open('./images_adv_PerC_AL/'+image_id_list[k*batch_size+i]+'.png'))
    label_adv=torch.argmax(model((X_adv-0.5)/0.5),dim=1)

#     # untargeted
#     label_ori=torch.argmax(model((X_ori-0.5)/0.5),dim=1)
#     cnt=cnt+torch.sum(label_adv!=label_ori)
    # targeted
    label_tar=torch.tensor(label_tar_list[k*batch_size:k*batch_size+batch_size_cur]).to(device)
    cnt=cnt+torch.sum(label_adv==label_tar)

print(cnt/len(image_id_list))


In [ ]:
## Evaluation on robustness

def bit_depth_red(X_before,depth):
    r=256/(2**depth)
    x_quan=torch.round(X_before*255/r)*r/255 
    return x_quan

def JPEG_compression(X_before,quality):
        X_after=torch.zeros_like(X_before)
        for j in range(X_after.size(0)):
            x_np=transforms.ToPILImage()(X_before[j].detach().cpu())
            x_np.save('./'+'j.jpg',quality=quality)
            X_after[j]=trn(Image.open('./'+'j.jpg'))
        return X_after
    

batch_size=50
num_batches = np.int(np.ceil(len(image_id_list)/batch_size))
levels=[90,80,70,60,50,40,30]#JPEG compression ratios
# levels=[7,6,5,4,3,2]#bit depths 
num=torch.zeros(len(levels))
for k in tqdm_notebook(range(0,num_batches)):
    batch_size_cur=min(batch_size,1000-k*batch_size)
    X_ori = torch.zeros(batch_size_cur,3,299,299).to(device) 
    X_adv_before=torch.zeros_like(X_ori)
    for i in range(batch_size_cur): 
        X_ori[i]=trn(Image.open('./images/'+image_id_list[k*batch_size+i]+'.png'))
        X_adv_before[i]=trn(Image.open('./images_adv_PerC_AL/'+image_id_list[k*batch_size+i]+'.png'))

    label_ori=torch.argmax(model((X_ori-0.5)/0.5),dim=1)
    label_adv_before=torch.argmax(model((X_adv_before-0.5)/0.5),dim=1)

    for i,value in enumerate(levels):
#         X_adv_after=bit_depth_red(X_adv_before,value)
        X_adv_after=JPEG_compression(X_adv_before,value)
        for k in range(1):
            label_after=torch.argmax(model((X_adv_after-0.5)/0.5),dim=1)
            num[i]=num[i]+torch.sum(label_after!=label_ori)

print(num/len(image_id_list))
    

In [ ]:
## Evaluation on tranferability

model2 = models.googlenet(pretrained=True,transform_input=False).eval()
for param in model2.parameters():
    param.requires_grad=False
device  = torch.device("cuda:0")
model2.to(device)

model3 = models.vgg16_bn(pretrained=True).eval()
for param in model3.parameters():
    param.requires_grad=False
device  = torch.device("cuda:0")
model3.to(device)

model4 = models.resnet152(pretrained=True).eval()
for param in model4.parameters():
    param.requires_grad=False
device  = torch.device("cuda:0")
model4.to(device)


#find eligible images for tranferability
batch_size=10
num_batches = np.int(np.ceil(len(image_id_list)/batch_size))
eligible_list=[]
for k in tqdm_notebook(range(0,num_batches)):
    batch_size_cur=min(batch_size,len(image_id_list)-k*batch_size)
    X_ori = torch.zeros(batch_size_cur,3,299,299).to(device) 
    for i in range(batch_size_cur): 
        X_ori[i]=trn(Image.open('./images/'+image_id_list[k*batch_size+i]+'.png'))

    pre_ori_1=torch.argmax(model((X_ori-0.5)/0.5),dim=1)
    pre_ori_2=torch.argmax(model2((X_ori-0.5)/0.5),dim=1)
    pre_ori_3=torch.argmax(model3(norm(X_ori)),dim=1)
    pre_ori_4=torch.argmax(model4(norm(X_ori)),dim=1)
    mask=((pre_ori_1==pre_ori_2).float() * (pre_ori_1==pre_ori_3).float() * (pre_ori_1==pre_ori_4).float() * (pre_ori_2==pre_ori_3).float() * (pre_ori_2==pre_ori_4).float() * (pre_ori_3==pre_ori_4).float())
    eligible_list.append(mask)
image_transfer_list=[]
for i in range(len(eligible_list)):
    for j in range(batch_size):
        if eligible_list[i][j]==1:
            image_transfer_list.append(image_id_list[i*batch_size+j])

#calculate tranferability
num_batches = np.int(np.ceil(len(image_transfer_list)/batch_size))
eligible_list=[]
adv_folder='./images_cw_untar_9000_1_k_40/'
sr_1=0
sr_2=0
sr_3=0
sr_4=0

for k in tqdm_notebook(range(0,num_batches)):
    batch_size_cur=min(batch_size,len(image_transfer_list)-k*batch_size)
    X_ori = torch.zeros(batch_size_cur,3,299,299).to(device) 
    x_adv = torch.zeros(batch_size_cur,3,299,299).to(device) 
    for i in range(batch_size_cur): 
        X_ori[i]=trn(Image.open('./images/'+image_transfer_list[k*batch_size+i]+'.png'))
        x_adv[i]=trn(Image.open(adv_folder+image_transfer_list[k*batch_size+i]+'.png'))
    pre_ori=torch.argmax(model((X_ori-0.5)/0.5),dim=1)
    
    pre_adv=torch.argmax(model((x_adv-0.5)/0.5),dim=1)
    pre_adv_2=torch.argmax(model2((x_adv-0.5)/0.5),dim=1)
    pre_adv_3=torch.argmax(model3(norm(X_ori)),dim=1)
    pre_adv_4=torch.argmax(model4(norm(X_ori)),dim=1)
    sr_1=sr_1+sum((pre_ori!=pre_adv).float())
    sr_2=sr_2+sum((pre_ori!=pre_adv_2).float())
    sr_3=sr_3+sum((pre_ori!=pre_adv_3).float())
    sr_4=sr_4+sum((pre_ori!=pre_adv_4).float())
print(sr_1/len(image_transfer_list),sr_2/len(image_transfer_list),sr_3/len(image_transfer_list),sr_4/len(image_transfer_list))


